Tutorial from: https://www.youtube.com/watch?v=q9MD_hU2Yd8

In [1]:
MODEL_NAME = "llama3.1"

## 1. Load data
We first load the data

In case we want to load 1 document only:

In [2]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

In [3]:
all_pages = []

pdf_files = list(Path('docs').iterdir())

for pdf in pdf_files:
    loader = PyPDFLoader(pdf)
    pages = loader.load()
    all_pages.extend(pages)  # Combine pages from all PDFs

In [4]:
"""
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('docs/gaussian_process.pdf')
pages = loader.load()
"""

"\nfrom langchain_community.document_loaders import PyPDFLoader\n\nloader = PyPDFLoader('docs/gaussian_process.pdf')\npages = loader.load()\n"

## 2. Split data

Because the context window of a LLM is limited we need to break down the documents into chunks.

We split such that every chunk uses a little bit of content of the previous chunk. We do that to avoid the model having incomplete information in each chunk.

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=100
)

In [7]:
split_texts = []
for doc in all_pages:
    chunks = splitter.split_documents([doc])  # Split each document separately
    split_texts.extend(chunks)  # Combine the split chunks

In [8]:
len(split_texts)

90

## 3. Store data

We use a VectorDB to store the chunks. We previously have to generate embeddings for them.

In [9]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

### 3.1. Generate embeddings

In [10]:
embeddings = OllamaEmbeddings(model=MODEL_NAME)

### 3.2. Store embeddings in ChromaDB

We first create the client of Ollama

In [11]:
vectorstore = Chroma.from_documents(split_texts, embeddings)

## 4. Retriever

How do I get data from my Vector store? We need to create a retriever object.

The retriever transforms the question into a vector and retrieves similar chunks from the vector store.

In [12]:
retriever = vectorstore.as_retriever()

Relevant chunks for the passed query

In [13]:
retriever.invoke("Normal distribution")

[Document(metadata={'page': 3, 'source': 'docs/g_process.pdf'}, page_content='“Regularization and Model Selection”).\nIn the case of Bayesian linear regression, however, the inte grals actually are tractable! In\nparticular, for Bayesian linear regression, one can show (a fter much work!) that\nθ|S∼ N(1\nσ2A−1XT⃗ y, A−1)\ny∗|x∗, S∼ N(1\nσ2xT\n∗A−1XT⃗ y, xT\n∗A−1x∗+σ2)\n4'),
 Document(metadata={'page': 12, 'source': 'docs/g_process.pdf'}, page_content='p(xA|xB) =1∫\nxAp(xA, xB;µ,Σ)dxA·[1\n(2π)m/2|Σ|1/2exp(\n−1\n2(x−µ)TΣ−1(x−µ))]\n(4)\n=1\nZ1exp{\n−1\n2([xA−µA\nxB−µB])T[VAAVAB\nVBAVBB][xA−µA\nxB−µB]}\n(5)\n13'),
 Document(metadata={'page': 10, 'source': 'docs/g_process.pdf'}, page_content='1. As Bayesian methods, Gaussian process models allow one to quantify uncertainty in\npredictions resulting not just from intrinsic noise in the p roblem but also the errors\nin the parameter estimation procedure. Furthermore, many m ethods for model selec-\ntion and hyperparameter selection in Bayesia

## 5. Configure a model

In [14]:
from langchain_ollama import ChatOllama

Temperature is basically telling the model how creative it needs to be. Zero is lowest?

In [15]:
model = ChatOllama(model=MODEL_NAME, temperature=0)

In [16]:
model.invoke("Who is the president of Spain?")

AIMessage(content="The President of Spain is currently Pedro Sánchez. He has been serving as Prime Minister (not President, actually) since June 2018 and leader of the Spanish Socialist Workers' Party (PSOE). However, the head of state in Spain is a monarch, known as the King or Queen of Spain.\n\nAs of my last update, the current monarch is King Felipe VI. He has been serving as the King of Spain since June 2014, following the abdication of his father, King Juan Carlos I.\n\nSo, to clarify:\n\n* The head of government (Prime Minister) in Spain is Pedro Sánchez.\n* The head of state (King or Queen) in Spain is King Felipe VI.", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-09-29T22:51:40.589182375Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2484382511, 'load_duration': 25705161, 'prompt_eval_count': 17, 'prompt_eval_duration': 41256000, 'eval_count': 144, 'eval_duration': 2376099000}, id=

### 5.1. Parsing model response

We do not consume the answer from the model. Instead, we sent it to a parser first


In [17]:
from langchain_core.output_parsers import StrOutputParser

In [18]:
parser = StrOutputParser()

We pipe the model with the parser

In [19]:
chain = model | parser

In [20]:
chain.invoke("Who is the prime minister of Spain?")

"As of my last update in April 2023, Pedro Sánchez Pérez-Castejón has been serving as the Prime Minister (Presidente del Gobierno) of Spain since June 2018. He leads a government that is part of the Spanish Socialist Workers' Party (PSOE). However, please note that political leadership can change over time due to elections or other factors. For the most current information, I recommend checking reputable news sources or official government websites for updates on the Prime Minister of Spain."

### 5.2. Setting up a prompt

In addition to the question we want to ask, we also provide the context from the documents. We are going to use a PromptTemplate do to so

In [21]:
from langchain.prompts import PromptTemplate

In [22]:
template = """
You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [23]:
print(prompt.format(context="Here is some context", question="Here is a question"))


You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: Here is some context

Question: Here is a question



We add the prompt to the chain

In [24]:
chain = prompt | model | parser

In [25]:
chain.invoke({
    "context": "Anna's sister is Susan", 
    "question": "Who is Susan's sister?"
})

'Anna.'

## 6. Final App

Putting all together, we need to add the retriever to the chain (pipe). The retriever will get the question and it will output a context (obtained from the vector store) for the prompt.

In [26]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [27]:
questions = [
    "Explain Gaussian processes in a nutshell",
    "Why do we use a Normal distribution?",
    "What does the covariance matrix has to do with kernels?"
]

In [28]:
answers = []
for question in questions:
    answer = chain.invoke(
        {'question': question}
    )
    answers.append(answer)
    print(answer)
    print("*" * 30)

A flexible framework for regression that models uncertain observations and can be used to perform tasks like linear regression, numerical integration, and solving differential equations. It's based on multivariate Gaussian distributions and can be adapted for different purposes.
******************************
Because it's based on the notion of the Gaussian distribution (normal distribution), named after Carl Friedrich Gauss.
******************************
The covariance matrix parameter is the Gram matrix of your N points with some desired kernel.
******************************


## Summary
1. Load data and split it into chunks.
2. Save chunks into a VectorDB:
    2.1. Generate embeddings.
    2.2. Create a retriever for the VectorDB.
3. Create a model.
4. Create parser.
5. Create a template prompt and a prompt.
6. Create a pipe with everything.